In [1]:
from smart_open import open
import json
import pandas as pd
import numpy as np

from importlib import reload
import docking_utils
reload(docking_utils)

<module 'docking_utils' from '/home/ubuntu/efs/mnt/efs0/alphafold/single_pass/docking_utils.py'>

In [2]:
receptor_file = "../code/uniprot_singlePass_transmembrane_receptors_simplified.tsv"
ligand_files = [f"../peptides/{lf}" for lf in [
    "20240812_peptidomics_wave5.json",
    "20240812_round_robin_portfolio.json",
    "20240905_receptor_ligand_prediction_peptidomics_run2.json"]]

In [3]:
receptor_names = []
limit = 300
for row in pd.read_csv(receptor_file, index_col=0).iterrows():
    receptor_names.append(row[1]["Entry Name"])
    
    limit -= 1
    if limit <= 0:
        break

ligand_names = []
ligands = []
for ligand_file in ligand_files:
    for ligand, coordinates in json.load(open(ligand_file)).items():
        for ligand_start, ligand_end in coordinates:
            ligand_names.append(f"{ligand}_{ligand_start}_{ligand_end}")
            ligands.append([ligand, ligand_start, ligand_end])

ranking_confidence_scores = []
for receptor in receptor_names:
    current_ranking_confidence_scores = []
    for ligand, ligand_start, ligand_end in ligands:
        experiment_name = f"{receptor}_{ligand}_{ligand_start}_{ligand_end}_13982528712754417"
        try:
            with open(f"s3://velia-af2-dev/outputs/{experiment_name}/{experiment_name}.model_1_multimer_v3_pred_0.scores.json") as f:
                ranking_confidence = json.load(f)["ranking_confidence"]
        except:
            ranking_confidence = -np.inf
        current_ranking_confidence_scores.append(ranking_confidence)
    ranking_confidence_scores.append(current_ranking_confidence_scores)

ranking_confidence_df = pd.DataFrame(ranking_confidence_scores, index=receptor_names, columns=ligand_names)

In [4]:
len(ranking_confidence_scores)

300

In [5]:
ranking_confidence_df

,GRP_HUMAN_95_128,GRP_HUMAN_103_139,GRP_HUMAN_83_93,SCG1_HUMAN_293_323,SCG1_HUMAN_421_437,SCG1_HUMAN_540_559,PACA_HUMAN_112_129,PACA_HUMAN_132_144,CMGA_HUMAN_342_355,CMGA_HUMAN_380_390,...,VTX-0862113_18_45,VTX-0862208_25_60,VTX-0863834_28_48,VTX-0864359_158_170,VTX-0864497_95_138,VTX-0864497_103_138,VTX-0865598_81_126,VTX-0865977_91_104,VTX-0868925_152_194,VTX-0869181_11_38
CD3CH_HUMAN,0.263405,0.280934,0.403763,0.215003,0.401285,0.303440,0.324846,0.385153,0.412655,0.340647,...,0.266223,0.313601,0.377326,0.422348,0.292315,0.307356,0.209030,0.329329,0.255009,0.340908
CIROP_HUMAN,0.291118,0.346281,0.278124,0.342128,0.617118,0.345328,0.286376,0.301709,0.255662,0.503078,...,0.253251,0.353806,0.445583,0.405061,0.306724,0.546729,0.244515,0.292596,0.333434,0.400611
T131L_HUMAN,0.286671,0.257856,0.269109,0.193516,0.189765,0.202338,0.263233,0.339007,0.217451,0.191016,...,0.230693,0.275281,0.236710,0.400750,0.211189,0.272639,0.200540,0.223314,0.299402,0.174278
MEG11_HUMAN,0.154381,0.158208,0.161664,0.170698,0.182134,0.239837,0.202928,0.215057,0.186022,0.162252,...,0.152846,0.192892,0.158815,0.269368,0.164150,0.178662,0.187034,0.177752,0.180553,0.185068
TIKI2_HUMAN,0.490210,0.437173,0.605553,0.415332,0.665690,0.556872,0.600885,0.577676,0.608695,0.598462,...,0.299600,0.408984,0.641255,0.878341,0.363806,0.402535,0.582431,0.738390,0.409828,0.599686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CD34_HUMAN,0.305664,0.291595,0.245968,0.179532,0.339555,0.219731,0.224759,0.242908,0.208134,0.181035,...,0.177986,0.209438,0.308477,0.294871,0.310496,0.333004,0.163969,0.355067,0.261406,0.306089
CD38_HUMAN,0.446579,0.468523,0.605759,0.508083,0.481604,0.501262,0.465079,0.534143,0.592300,0.584918,...,0.361222,0.341908,0.440317,0.666903,0.764153,0.385151,0.404398,0.487069,0.367686,0.404830
TNR8_HUMAN,0.265847,0.264050,0.415107,0.264557,0.316445,0.296442,0.302674,0.373671,0.411779,0.249618,...,0.268464,0.278556,0.313399,0.276167,0.290236,0.305917,0.220740,0.314666,0.241339,0.284220
CD1B_HUMAN,0.419344,0.413667,0.593694,0.334742,0.500524,0.420778,0.498238,0.615050,0.584310,0.556924,...,0.455960,0.463609,0.465520,0.544039,0.375507,0.400073,0.373472,0.521809,0.389082,0.461003


In [19]:
high_conf_pairs = []
for row in ranking_confidence_df.iterrows():
    receptor = row[0]
    for ligand, confidence_score in row[1].items():
        if confidence_score > 0.80:
            high_conf_pairs.append((receptor, ligand, confidence_score))
            print(f"{receptor}_{ligand}_13982528712754417\t{confidence_score}")
print(high_conf_pairs)

TIKI2_HUMAN_GHRL_HUMAN_24_37_13982528712754417	0.8113800625359099
TIKI2_HUMAN_VTX-0085107_21_36_13982528712754417	0.8013808336376894
TIKI2_HUMAN_VTX-0315807_24_48_13982528712754417	0.8326330000182448
TIKI2_HUMAN_VTX-0864359_158_170_13982528712754417	0.8783409053416216
GGT3_HUMAN_CMGA_HUMAN_342_355_13982528712754417	0.8005155796165078
GGT3_HUMAN_APOA2_HUMAN_22_44_13982528712754417	0.898450572582308
LIRA5_HUMAN_VTX-0859974_20_28_13982528712754417	0.8074618164432669
KLRF2_HUMAN_VTX-0864359_158_170_13982528712754417	0.8414201878579649
TGR3L_HUMAN_CMGA_HUMAN_342_355_13982528712754417	0.8422589763479857
TGR3L_HUMAN_SCG2_HUMAN_300_316_13982528712754417	0.83064649767158
TMPS2_HUMAN_GHRL_HUMAN_24_37_13982528712754417	0.814499228132871
FLRT2_HUMAN_GLUC_HUMAN_53_68_13982528712754417	0.8478006323039448
ADA20_HUMAN_NUCB1_HUMAN_73_82_13982528712754417	0.8082080247329149
ADA20_HUMAN_VTX-0331059_19_32_13982528712754417	0.8160525385889507
ADA20_HUMAN_VTX-0859974_20_28_13982528712754417	0.86610183661964

In [42]:
pdb = docking_utils.load_s3_pdb('HEPS_HUMAN_VTX-0859974_16_28_13982528712754417')
docking_utils.view_pdb(pdb)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
pdb = docking_utils.load_s3_pdb('HLAB_HUMAN_NPW_HUMAN_76_84_13982528712754417')
docking_utils.view_pdb(pdb)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [9]:
pdb = docking_utils.load_s3_pdb('FLRT2_HUMAN_GLUC_HUMAN_53_68_13982528712754417')
docking_utils.view_pdb(pdb)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.